# 04 — Asset Allocation Backtest

Map regime probabilities to portfolio weights and backtest against
buy-and-hold and 60/40 benchmarks.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src.allocation.regime_allocator import RegimeAllocator
from src.allocation.backtester import Backtester

In [ ]:
# Example: create synthetic regime probs and asset returns
import numpy as np
rng = np.random.default_rng(42)
T = 120
dates = pd.date_range('2014-01-31', periods=T, freq='ME')

# Synthetic regime probabilities
raw_probs = rng.dirichlet([4, 2, 1, 2], size=T)
regime_probs = pd.DataFrame(
    raw_probs, index=dates,
    columns=['expansion', 'slowdown', 'recession', 'recovery']
)

# Synthetic asset returns
asset_returns = pd.DataFrame({
    'equities':    rng.normal(0.008, 0.04, T),
    'bonds':       rng.normal(0.003, 0.015, T),
    'commodities': rng.normal(0.005, 0.05, T),
    'cash':        rng.normal(0.001, 0.001, T),
}, index=dates)

print('Regime probs:', regime_probs.shape)
print('Asset returns:', asset_returns.shape)

In [ ]:
bt = Backtester()
result = bt.run(regime_probs, asset_returns)
benchmarks = bt.run_benchmarks(asset_returns)

print('=== Regime Strategy ===')
print(result.summary())
print()
for name, bench in benchmarks.items():
    print(f'=== {name.replace("_", " ").title()} ===')
    print(bench.summary())
    print()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
result.equity_curve.plot(ax=ax, label='Regime Strategy', linewidth=2)
for name, bench in benchmarks.items():
    bench.equity_curve.plot(ax=ax, label=name.replace('_', ' ').title(), linestyle='--')
ax.set_title('Portfolio Equity Curves')
ax.set_ylabel('Cumulative Value')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()